# Лекция 2.3: Метрические алгоритмы и наивный байесовский классификатор

## Метод ближайших соседей

Обычно с метрическими алгоритмами знакомятся на примере частного случая – **kNN (k ближайших соседей)**. Суть данного метода заключается в том, что на основании известных признаков для объекта находятся соседние, располагающиеся как можно ближе к исходному объекту в некотором пространстве признаков. Затем по этим объектам вычисляется значение недостающего (обычно целевого) признака.

Возникает два вопроса:
- Каким образом представить объекты в пространстве и что это должно быть за пространство?
- Как вычислять расстояние между ними?

Ответ на оба этих вопроса зачастую зависит от задачи. Объект обычно проще всего представить как вектор признаков, а для подсчета расстояния используются, например:
- евклидово расстояние или расстояние Минковского
- косинусное расстояние
- манхэттенское расстояние
- расстояние Чебышёва

> Какую именно метрику использовать в конкретной задаче, обычно можно определить только экспериментально

> На практике бывает полезно использовать не сам kNN, а некоторые его приближенные альтернативы. Их реализация представлена, например, в библиотеке векторной базы [FAISS](https://github.com/facebookresearch/faiss).

Опишем задачу формально. Пусть задана обучающая выборка пар:
$$X^m = \{(x_1,y_1),\dots,(x_m,y_m)\}$$

Пусть на множестве объектов задана функция расстояния $\rho(x,x')$. Чем больше значение этой функции, тем менее схожими являются два объекта $x, x'$.

В простейшем случае используется евклидова метрика:
$$\rho(x,x') = \sqrt {\sum _{i=1}^{n}(x_{i}-x'_{i})^{2}}$$

Для произвольного объекта $u$ расположим объекты обучающей выборки $x_i$ в порядке возрастания расстояний до $u$:
$$\rho(u,x_{1; u}) \leq  \rho(u,x_{2; u}) \leq \cdots \leq \rho(u,x_{m; u})$$

где через $x_{i; u}$ обозначается тот объект обучающей выборки, который является $i$-м соседом объекта $u$.

Аналогичное обозначение введём и для ответа на $i$-м соседе: $y_{i; u}$.

Таким образом, произвольный объект $u$ порождает свою перестановку выборки. В наиболее общем виде алгоритм ближайших соседей есть:
$$a(u) = \mathrm{arg}\max_{y\in Y} \sum_{i=1}^k \bigl[ y_{i; u}=y \bigr] w(i,u),$$

где $w(i,u)$ – заданная весовая функция, которая оценивает степень важности $i$-го соседа для классификации объекта $u$.

Естественно полагать, что эта функция не отрицательна и не возрастает по $i$ (поскольку чем дальше объект, тем меньший вклад он должен вносить в пользу своего класса).

По-разному задавая весовую функцию, можно получать различные варианты метода ближайших соседей:
- $w(i,u) = [i=1]$ – простейший метод ближайшего соседа;
- $w(i,u) = [i\leq k]$ – метод $k$ ближайших соседей;
- $w(i,u) = [i\leq k] q^i$ – метод $k$ экспоненциально взвешенных ближайших соседей, где предполагается константа $q < 1$

Для избежания неоднозначности ответа при классификации в качестве весовой функции $w(i, u)$ используют функцию ядра сглаживания $K(r)$

Примеры ядер:
- Треугольное: \quad ${\displaystyle K(r) = 1 - |r|}$
- Параболическое: \quad ${\displaystyle K(r) = \frac{3}{4}(1 - r^2)}$
- Трёхкубическое: \quad ${\displaystyle K(r) = \frac{70}{81}(1 - |r|^3)^3}$

Алгоритм $k$ ближайших соседей можно обобщить с помощью функции ядра. Рассмотрим два способа, которыми это можно сделать.

Метод парзеновского окна фиксированной ширины  $h$:
$$w(i,u) = K\biggl(\frac{\rho(u,x_{i; u})}{h}\biggr)$$

Метод парзеновского окна переменной ширины:
$$w(i,u) = K\biggl(\frac{\rho(u,x_{i; u})}{\rho(u,x_{k+1; u})}\biggr)$$

Таким образом классификаторы, полученные при использовании этих методов, можно записать в следующем виде

Фиксированной ширины: $$a_h = \mathrm{arg}\max_{y\in Y} \sum_{i=1}^k \bigl[ y_{i; u}=y \bigr] K\biggl(\frac{\rho(u,x_{i; u})}{h}\biggr)$$

Переменной ширины (фиксированное число соседей):
$$a_k = \mathrm{arg}\max_{y\in Y} \sum_{i=1}^k \bigl[ y_{i; u}=y \bigr] K\biggl(\frac{\rho(u,x_{i; u})}{\rho(u,x_{k+1; u})}\biggr)$$

Практический пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
X, y = make_blobs(n_samples=200, centers=2, cluster_std=10, random_state=42)

k = 3
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X, y)

x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

In [ ]:
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.colors.ListedColormap(['#FFDD2D', '#428BF9']))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.colors.ListedColormap(['#FFDD2D', '#428BF9']),
            s=50, edgecolors='white', linewidth=0.5)
plt.title(f'kNN (k={k})')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

## Наивный байесовский классификатор

Рассмотрим алгоритм наивного байесовского классификатора – это простой, но мощный метод классификации, основанный на **теореме Байеса** в предположении о независимости признаков при заданном классе.

Вспомним, что теорема Байеса позволяет вычислить апостериорную вероятность класса $y$ при наличии признаков $X_1, X_2, ..., X_p$:
$$P(y|X_1, X_2, ..., X_p) = \frac{P(X_1, X_2, ..., X_p|y) \cdot P(y)}{P(X_1, X_2, ..., X_p)}$$

- $P(y|X_1, X_2, ..., X_p)$ – вероятность того, что объект принадлежит классу \( y \) при данных признаках.
- $P(X_1, X_2, ..., X_p|y)$ – вероятность наблюдать данные признаки при условии, что объект принадлежит классу $y$.
- $P(y)$ – априорная вероятность класса $y$.
- $P(X_1, X_2, ..., X_p)$ – вероятность наблюдать данные признаки (нормировочная константа).

Классификатор называется **наивным**, потому что делает сильное (и такое еще ни раз встретится) предположение о независимости признаков при заданном классе:
$$P(X_1, X_2, ..., X_p|y) = \prod_{i=1}^p P(X_i|y)$$

Это предположение упрощает вычисления, но редко выполняется в реальных данных. Тем не менее, на практике алгоритм часто работает хорошо даже при его нарушении.

Для используется для упрощения вычислений, особенно при работе с малыми вероятностями и перехода от перемножения вероятностей, которые могут быть очень маленькими, к их сложению используется логарифмирование (этот переход тоже еще ни раз встретится):
$$\ln P(y|X_1, X_2, ..., X_p) \propto \ln P(y) + \sum_{i=1}^p \ln P(X_i|y)$$

Если в обучающей выборке не встречается какое-то значение признака для класса, то $P(X_i|y) = 0$, и произведение вероятностей станет нулевым, что приведёт к неверной классификации. Для решения этой проблемы используется **сглаживание** (например, сглаживание Лапласа или Лидстоуна). Оно добавляет небольшое значение $\alpha$ к каждому счёту:
$$P(X_i|y) = \frac{\text{количество объектов с } X_i \text{ в классе } y + \alpha}{\text{общее количество объектов в классе } y + \alpha \cdot n}$$

где $n$ — количество возможных значений признака $X_i$.

В итоге, алгоритм наивного байесовского классификатора можно записать следующим образом:
1. Обучение:
   - Вычислить априорные вероятности классов $P(y)$.
   - Вычислить условные вероятности признаков для каждого класса $P(X_i|y)$.
2. Предсказание:
   - Для нового объекта с признаками $X_1, X_2, ..., X_p$ вычислить для каждого класса $y$: $F(y) = \ln P(y) + \sum_{i=1}^p \ln P(X_i|y).$
   - Выбрать класс $y^*$, для которого $F(y)$ максимально: $y^* = \arg\max_{y \in Y} F(y).$

Практический пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.pipeline import make_pipeline

In [ ]:
categories = [
    'rec.autos',
    'comp.graphics',
    'sci.space',
    'rec.sport.baseball'
]

newsgroups = fetch_20newsgroups(
    subset='all',
    categories=categories,
    remove=('headers', 'footers', 'quotes'),
    shuffle=True,
    random_state=42
)

X = newsgroups.data
y = newsgroups.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [ ]:
pipeline = make_pipeline(
    CountVectorizer(
        max_features=5000,
        stop_words='english',
        min_df=2,
        max_df=0.8
    ),
    MultinomialNB(alpha=1.0)
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"{accuracy:.4f}")

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm,
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=newsgroups.target_names,
            yticklabels=newsgroups.target_names)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()